#  Setup stuff:  Connectivity 

In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


In [ ]:

from katana import remote
#  from katana.remote import import_data

my_client=remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


#  Sample Cyphers: Vertices

In [ ]:

#  Cypher clauses
#
#     RETURN ..
#     ORDER BY ..
#     SKIP ..
#     LIMIT ..


In [ ]:

#  Use of some of the traversal methods ..
#

l_query  = """

   MATCH (n) - [r] -> (m)
   RETURN
      LABELS(m)         AS my_labels,
      ID(m)             AS my_id,
      KEYS(m)           AS my_keys,
      properties(m)     AS my_properties

   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+----------------+-----------------+---------------------------------+-----------------------+------------------------------+--------------------+---------------------------------+
#     |    | my_labels      |           my_id | my_keys                         | my_properties.LABEL   | my_properties.airport_name   | my_properties.id   | my_properties.restaurant_name   |
#     |----+----------------+-----------------+---------------------------------+-----------------------+------------------------------+--------------------+---------------------------------|
#     |  0 | ['Airport']    |               0 | ['LABEL', 'airport_name', 'id'] | Airport               | Milwaukee                    | MKE                | nan                             |
#     |  1 | ['Airport']    | 562949953421312 | ['LABEL', 'airport_name', 'id'] | Airport               | Denver                       | DEN                | nan                             |
#     |  2 | ['Airport']    |               1 | ['LABEL', 'airport_name', 'id'] | Airport               | Chicago O-Hare               | ORD                | nan                             |
#     |  3 | ['Airport']    |               1 | ['LABEL', 'airport_name', 'id'] | Airport               | Chicago O-Hare               | ORD                | nan                             |
#     |  4 | ['Airport']    | 562949953421312 | ['LABEL', 'airport_name', 'id'] | Airport               | Denver                       | DEN                | nan                             |
#     |  5 | ['Restaurant'] | 281474976710657 | ['id', 'restaurant_name']       | nan                   | nan                          | PAP                | Pappadeauxs Seafood Kitchen     |
#     |  6 | ['Airport']    | 281474976710656 | ['LABEL', 'airport_name', 'id'] | Airport               | San Jose                     | SJC                | nan                             |
#     |  7 | ['Airport']    |               1 | ['LABEL', 'airport_name', 'id'] | Airport               | Chicago O-Hare               | ORD                | nan                             |
#     +----+----------------+-----------------+---------------------------------+-----------------------+------------------------------+--------------------+---------------------------------+


In [37]:

#  Average traversal; just nodes
#

l_query  = """
   MATCH (n) 
   RETURN 
      ID(n)               AS my_id, 
      LABELS(n)           AS my_label, 
      n.airport_name      AS airport_name, 
      n.restaurant_name   AS restaurant_name
   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+-----------------+----------------+----------------+-----------------------------+
#     |    |           my_id | my_label       | airport_name   | restaurant_name             |
#     |----+-----------------+----------------+----------------+-----------------------------|
#     |  0 |               1 | ['Airport']    | Chicago O-Hare |                             |
#     |  1 |               0 | ['Airport']    | Milwaukee      |                             |
#     |  2 | 281474976710656 | ['Airport']    | San Jose       |                             |
#     |  3 | 281474976710657 | ['Restaurant'] |                | Pappadeauxs Seafood Kitchen |
#     |  4 | 562949953421312 | ['Airport']    | Denver         |                             |
#     +----+-----------------+----------------+----------------+-----------------------------+


          0/? [?op/s]

+----+-----------------+----------------+----------------+-----------------------------+
|    |           my_id | my_label       | airport_name   | restaurant_name             |
|----+-----------------+----------------+----------------+-----------------------------|
|  0 |               1 | ['Airport']    | Chicago O-Hare |                             |
|  1 |               0 | ['Airport']    | Milwaukee      |                             |
|  2 | 281474976710656 | ['Airport']    | San Jose       |                             |
|  3 | 281474976710657 | ['Restaurant'] |                | Pappadeauxs Seafood Kitchen |
|  4 | 562949953421312 | ['Airport']    | Denver         |                             |
+----+-----------------+----------------+----------------+-----------------------------+


In [38]:

#  Get a count of all nodes

display(my_graph.num_nodes())


l_query  = """
   MATCH (n) 
   RETURN COUNT(*) AS cnt
   """.format()

l_count = my_graph.query(l_query)["cnt"][0]


print(l_count)
print(type(l_count))


          0/? [?op/s]

5

          0/? [?op/s]

5
<class 'numpy.int64'>


In [39]:

#  Get a count of all node types

l_query  = """
   MATCH (n) 
   RETURN COUNT(DISTINCT LABELS(n)) AS cnt
   """.format()

l_count = my_graph.query(l_query)["cnt"][0]

print(l_count)


          0/? [?op/s]

2


In [40]:

#  List all node types

l_query  = """
   MATCH (n)
   RETURN DISTINCT LABELS(n) AS my_type
   """.format()

l_result = my_graph.query(l_query)


for l_each in l_result.itertuples():
   print(l_each.my_type)

print("--")


          0/? [?op/s]

['Restaurant']
['Airport']
--


In [41]:

#  List all node types with node counts

l_query  = """
   MATCH (n) 
   WITH DISTINCT LABELS(n) AS temp, COUNT(n) AS tempCnt
   UNWIND temp AS label
   RETURN label, SUM(tempCnt) AS cnt
   ORDER BY label
   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))

    
print("")


l_query  = """
   MATCH (n) 
   WITH COUNT(n) AS tempCnt
   RETURN tempCnt
   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))



          0/? [?op/s]

+----+------------+-------+
|    | label      |   cnt |
|----+------------+-------|
|  0 | Airport    |     4 |
|  1 | Restaurant |     1 |
+----+------------+-------+



          0/? [?op/s]

+----+-----------+
|    |   tempCnt |
|----+-----------|
|  0 |         5 |
+----+-----------+


#  Sample Cyphers: Edges

In [ ]:

#  Count all edges

display(my_graph.num_edges())

l_query  = """
    MATCH (n) - [r] -> (m)
    RETURN COUNT(r) AS cnt
   """.format()

l_count = my_graph.query(l_query)["cnt"][0]

print(l_count)


In [42]:

#  List all vertice types with counts

l_query  = """
   MATCH (m)-[r]->(n) 
   WITH DISTINCT TYPE(r) AS temp, COUNT(r) AS tempCnt
   RETURN temp, tempCnt
   ORDER BY temp
   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


l_query  = """
   MATCH (m)-[r]->(n) 
   WITH COUNT(r) AS tempCnt
   RETURN tempCnt
   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+------------+-----------+
|    | temp       |   tempCnt |
|----+------------+-----------|
|  0 | ATTRACTION |         1 |
|  1 | FLIES_TO   |         7 |
+----+------------+-----------+


          0/? [?op/s]

+----+-----------+
|    |   tempCnt |
|----+-----------|
|  0 |         8 |
+----+-----------+


#  Sample Cyphers: Properties

In [ ]:

#  Return properties/keys from all vertices

my_graph.query("CALL graph.schema() RETURN *")


In [43]:

#  Return properties/keys from all vertices

l_query  = """
   MATCH (n)
   RETURN DISTINCT LABELS(n) AS label, KEYS(n)
   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+----------------+---------------------------------+
|    | label          | KEYS(n)                         |
|----+----------------+---------------------------------|
|  0 | ['Airport']    | ['LABEL', 'airport_name', 'id'] |
|  1 | ['Restaurant'] | ['id', 'restaurant_name']       |
+----+----------------+---------------------------------+


In [44]:

#  KAT-11443    throws cancelled error, should work
#
#     Work around in next cell

l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN DISTINCT TYPE(r) AS my_type, KEYS(r)
   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+------------+----------------------------------------------------------------------+
|    | my_type    | KEYS(r)                                                              |
|----+------------+----------------------------------------------------------------------|
|  0 | ATTRACTION | ['DRIVABLE']                                                         |
|  1 | FLIES_TO   | ['DISTANCE', 'NUM_HOPS', 'TYPE', 'DIST_MILES_0021', 'DIST_KMS_0021'] |
|  2 | FLIES_TO   | ['DISTANCE', 'NUM_HOPS', 'TYPE', 'DIST_MILES_0021', 'DIST_KMS_0021'] |
+----+------------+----------------------------------------------------------------------+


In [45]:

#  KAT-11445  Missing data, bad data
#  MAybe also  KAT-11448


#  Return properties/keys from all edges
#

l_query  = """
   MATCH (n) - [r] -> (m)
   // RETURN DISTINCT (r.TYPE) AS my_type, KEYS(r) AS my_keys
   RETURN DISTINCT TYPE(r) AS my_type, KEYS(r) AS my_keys
   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


#  Sample output,
#
#     +----+------------+----------------------------------------------------------------------+
#     |    | my_type    | my_keys                                                              |
#     |----+------------+----------------------------------------------------------------------|
#     |  0 | ATTRACTION | ['DRIVABLE']                                                         |
#     |  1 | FLIES_TO   | ['DISTANCE', 'NUM_HOPS', 'TYPE', 'DIST_MILES_0021', 'DIST_KMS_0021'] |
#     |  2 | FLIES_TO   | ['DISTANCE', 'NUM_HOPS', 'TYPE', 'DIST_MILES_0021', 'DIST_KMS_0021'] |
#     +----+------------+----------------------------------------------------------------------+


          0/? [?op/s]

+----+------------+----------------------------------------------------------------------+
|    | my_type    | my_keys                                                              |
|----+------------+----------------------------------------------------------------------|
|  0 | ATTRACTION | ['DRIVABLE']                                                         |
|  1 | FLIES_TO   | ['DISTANCE', 'NUM_HOPS', 'TYPE', 'DIST_MILES_0021', 'DIST_KMS_0021'] |
|  2 | FLIES_TO   | ['DISTANCE', 'NUM_HOPS', 'TYPE', 'DIST_MILES_0021', 'DIST_KMS_0021'] |
+----+------------+----------------------------------------------------------------------+


#  Sample Cyphers: Diagnostics

In [ ]:

#  Also at,
#     https://amusing-fawn.clusters-k9h.dev/v2/widgets/katana-frontend/databases

my_client.widgets().operations()

#  my_client.widgets().databases()
#  my_client.widgets().dashboard()                #  No-op currently


In [ ]:

# result.raw() went away
#
#  to pretty print a result as JSON, use ..

#  Just to get some data ..
#

l_query  = """
   MATCH (n) - [r] -> (m)
   RETURN DISTINCT type(r) AS type, keys(r)
   """.format()

l_result = my_graph.query(l_query)


import json

l_result_json = json.loads(l_result.to_json())
l_result_strn = json.dumps(l_result_json, indent=2)

print(l_result_strn)



In [47]:

#  Produce a visual graph

l_result = my_graph.query("""

   MATCH (n) -[r]-> (m)
   RETURN n, r, m
   LIMIT 1000 
   
   """,
   contextualize=True)

l_result.view()


          0/? [?op/s]

          0/? [?op/s]

#  More Cypher 

In [46]:
#  Parameterized query, works with (Airports) ..

l_fname = "MKE"

l_query  = """

   MATCH (n: Airport) 
   WHERE n.id = '{0}'
   RETURN n

   """.format(l_fname)

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+--------+----------+
|    |   n.internal_id | n.labels    | n.LABEL   | n.airport_name   | n.id   | n.type   |
|----+-----------------+-------------+-----------+------------------+--------+----------|
|  0 |               0 | ['Airport'] | Airport   | Milwaukee        | MKE    | node     |
+----+-----------------+-------------+-----------+------------------+--------+----------+


In [ ]:

#  Regex query, tested with (Airports) ..

l_fname = "/^M*/"

l_query  = """

   MATCH (n: Airport) 
   WHERE n.airport_name =~ '{0}'
   RETURN n.id, n.airport_name

   """.format(l_fname)

l_result = my_graph.query(l_query)

display(print(l_result[0:5]))

#  Sample output,
#
#     OperationError: FmPz7gHk5N3wKxVziuy6KiRd9D9U3KiqSCaursUBfDEG-4R77ceGCNHQJRCW56 backtrace (QueryOperation.cpp:67):
#     backtrace (Network.h:331): backtrace (QueryOperation.cpp:70): backtrace (QueryClient.cpp:603): unable to translate
#     openCypher parse tree to internal IR (OpGraph.cpp:224): string comparision with regular expression not supported
#     (CypherOpGraphBuilder.cpp:57): InternalError:NotImplemented: InternalError:NotImplemented: InternalError:NotImplemented



In [ ]:
# ###################################################################

In [ ]:

#  Changing the Edge type
#

l_query  = """

   MATCH (n) - [r] -> (m)
   RETURN r

   """.format()

l_result = my_graph.query(l_query)
   #
#  print(tabulate(l_result, headers='keys', tablefmt='psql'))

   ###

l_query  = """

   MATCH (n) - [r] -> (m)
   RETURN
      TYPE(r)           AS my_type,
      ID(r)             AS my_id,
      KEYS(r)           AS my_keys,
         //
      ID(startNode(r))  AS my_startnode_id,
      ID(endNode(r))    AS my_endnode_id

   """.format()

l_result = my_graph.query(l_query)
   #
print(tabulate(l_result, headers='keys', tablefmt='psql'))


#  Sample output,
#
#     +----+-----------------+----------------+-----------------+------------+--------------+-------------------+---------------------+--------------+----------+-----------------+------------------+----------+--------------+
#     |    |        r.end.id | r.end.labels   |   r.internal_id | r.label    |   r.DISTANCE |   r.DIST_KMS_0021 |   r.DIST_MILES_0021 |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   | r.DRIVABLE   |
#     |----+-----------------+----------------+-----------------+------------+--------------+-------------------+---------------------+--------------+----------+-----------------+------------------+----------+--------------|
#     |  0 |               1 | ['Airport']    |               1 | FLIES_TO   |           66 |               106 |                  66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
#     |  1 |               0 | ['Airport']    |               0 | FLIES_TO   |           66 |               106 |                  66 |            1 | FLIES_TO |               1 | ['Airport']      | edge     | nan          |
#     |  2 | 562949953421312 | ['Airport']    |               2 | FLIES_TO   |          886 |              1425 |                 886 |            1 | FLIES_TO |               1 | ['Airport']      | edge     | nan          |
#     |  3 |               1 | ['Airport']    | 281474976710656 | FLIES_TO   |         1829 |              2943 |                1829 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     | nan          |
#     |  4 | 562949953421312 | ['Airport']    | 281474976710657 | FLIES_TO   |          948 |              1525 |                 948 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     | nan          |
#     |  5 | 281474976710657 | ['Restaurant'] | 281474976710658 | ATTRACTION |          nan |               nan |                 nan |          nan | nan      | 281474976710656 | ['Airport']      | edge     | Yes          |
#     |  6 | 281474976710656 | ['Airport']    | 562949953421312 | FLIES_TO   |          948 |              1525 |                 948 |            1 | FLIES_TO | 562949953421312 | ['Airport']      | edge     | nan          |
#     |  7 |               1 | ['Airport']    | 562949953421313 | FLIES_TO   |          886 |              1425 |                 886 |            1 | FLIES_TO | 562949953421312 | ['Airport']      | edge     | nan          |
#     +----+-----------------+----------------+-----------------+------------+--------------+-------------------+---------------------+--------------+----------+-----------------+------------------+----------+--------------+
#
